In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from xgboost import XGBRegressor

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
TRAIN_VIEW = "WATER_QUALITY.FEATURES.TRAINING_FEATURES" 
VAL_VIEW = "WATER_QUALITY.FEATURES.VALIDATION_FEATURES" 
SUBMISSION_TEMPLATE = "WATER_QUALITY.RAW.SUBMISSION_TEMPLATE"
DATE_COL   = "SAMPLE_DATE"
ID_COLS    = ["SAMPLE_DATE", "LATITUDE", "LONGITUDE"]
DROP  = ["YEAR", "MONTH"]
TARGETS    = {
    "TOTAL_ALKALINITY": "total_alkalinity",
    "ELECTRICAL_CONDUCTANCE": "electrical_conductance",
    "DISSOLVED_REACTIVE_PHOSPHORUS": "dissolved_reactive_phosphorus"
}

In [ ]:
df = session.table(TRAIN_VIEW).to_pandas()
ho_df = session.table(VAL_VIEW).to_pandas()
submission_df = session.table(SUBMISSION_TEMPLATE).to_pandas()

In [ ]:
df.shape

In [ ]:
df = df.dropna()
df.shape

In [ ]:
def spatial_temporal_train_val_split(
    df: pd.DataFrame,
    date_col: str,
    lat_col: str,
    lon_col: str,
    spatial_holdout_frac: float = 0.2,
    time_train_frac: float = 0.8,
    cell_km: float = 25,
    seed: int = 42
):
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])

    # ---- Spatial blocks --------------------------------------
    lat = df[lat_col].astype(float).to_numpy()
    lon = df[lon_col].astype(float).to_numpy()

    deg_lat = cell_km / 111.0
    mean_lat_rad = np.deg2rad(np.nanmean(lat))
    deg_lon = cell_km / (111.0 * np.cos(mean_lat_rad) + 1e-12)

    lat_bin = np.floor(lat / deg_lat).astype(int)
    lon_bin = np.floor(lon / deg_lon).astype(int)

    df["spatial_block"] = list(zip(lat_bin, lon_bin))

    # ---- Spatial holdout -------------------------------------
    rng = np.random.default_rng(seed)
    blocks = df["spatial_block"].unique()
    n_holdout = max(1, int(len(blocks) * spatial_holdout_frac))
    holdout_blocks = set(rng.choice(blocks, size=n_holdout, replace=False))

    seen = df[~df["spatial_block"].isin(holdout_blocks)].copy()

    # ---- Time-aware split on seen blocks ---------------------
    seen = seen.sort_values(date_col)
    cut = int(len(seen) * time_train_frac)

    train = seen.iloc[:cut].copy().drop(columns=['spatial_block'])
    val   = seen.iloc[cut:].copy().drop(columns=['spatial_block'])
    
    print(f"train shape: {train.shape}")
    print(f"\n val shape: {val.shape}")
    
    return train, val

In [ ]:
# Normalize column names
df.columns = [c.lower() for c in df.columns]
ho_df.columns = [c.lower() for c in ho_df.columns]
submission_df.columns = [c.lower() for c in submission_df.columns]

date_col = DATE_COL.lower()

# Sort by time, do time-aware split (80/20)
df = df.sort_values(date_col)
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42)

In [ ]:
train_df.head()

# Identify feature columns

In [ ]:
target_cols = [v for v in TARGETS.values()]
drop_cols = set([c.lower() for c in ID_COLS] + target_cols + DROP)

feature_cols = [c for c in df.columns if c not in drop_cols]

# Separate numeric vs categorical

In [ ]:
numeric_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(df[c])]
categorical_cols = [c for c in feature_cols if pd.api.types.is_object_dtype(df[c])]

# Preprocess

In [ ]:
numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer()),
    ("scaler", StandardScaler())
])

categorical_pipe = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

pre = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, numeric_cols),
        ("cat", categorical_pipe, categorical_cols)
    ],
    remainder="drop"
)

In [ ]:
def train_one_target(y_col: str):
    # Drop rows where this target is null
    tr = train_df[train_df[y_col].notna()].copy()
    va = val_df[val_df[y_col].notna()].copy()

    X_train, y_train = tr[feature_cols], tr[y_col]
    X_val, y_val     = va[feature_cols], va[y_col]

    y_train = np.log1p(y_train.values)
    y_val = np.log1p(y_val.values)

    model = Ridge(alpha=1.0, random_state=42)

    pipe = Pipeline(steps=[
        ("preprocess", pre),
        ("model", model)
    ])

    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_val)

    mae = mean_absolute_error(y_val, pred)
    mse  = mean_squared_error(y_val, pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_val, pred)

    return pipe, {"mae": mae, "rmse": rmse, "r2": r2, "n_val": len(y_val)}

In [ ]:
models = {}
metrics = {}

for label, col in TARGETS.items():
    m, met = train_one_target(col)
    models[label] = m
    metrics[label] = met

metrics

In [ ]:
pipe = models["TOTAL_ALKALINITY"]
ridge = pipe.named_steps["model"]

coef = ridge.coef_
intercept = ridge.intercept_

feature_names = pipe.named_steps["preprocess"].get_feature_names_out()

coef_df = pd.DataFrame({
    "feature": feature_names,
    "coefficient": coef
}).sort_values("coefficient", key=abs, ascending=False)

coef_df.head(20)

In [ ]:
preds = {}

for target in target_cols:
    pipe = models[target.upper()]
    pred = pipe.predict(ho_df[feature_cols])
    preds[target] = np.clip(np.expm1(pred), 0, None)

In [ ]:
#ids = ["sample_date", "latitude", "longitude"]

#pred_df = ho_df[ids].copy()
#for target in target_cols:
    #pred_df[target] = preds[target]

#submission_df = submission_df.drop(columns=target_cols, errors="ignore").merge(pred_df, on=ids, how="left")

In [ ]:
#submission_df.columns = sub_cols
#submission_df.head(submission_df.shape[0])

In [ ]:
def fit_xgb_per_target(train_df, val_df, feature_cols, target_cols, seed=42):
    fitted = {}
    val_preds = {}
    metrics = []

    X_tr = train_df[feature_cols]
    X_va = val_df[feature_cols]

    for t in target_cols:
        y_tr = train_df[t].values
        y_val = val_df[t].values

        # Safety: enforce non-negativity for transform (if you have tiny negatives from noise)
        y_tr = np.clip(y_tr, 0, None)
        y_tr_t = np.log1p(y_tr)

        model = XGBRegressor(
            n_estimators=3000,
            learning_rate=0.03,
            max_depth=6,
            min_child_weight=5,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.0,
            reg_lambda=1.0,
            gamma=0.0,
            objective="reg:squarederror",
            tree_method="hist",
            random_state=seed,
            n_jobs=-1,
            early_stopping_rounds=100
        )

        model.fit(
            X_tr, y_tr_t,
            eval_set=[(X_va, np.log1p(np.clip(val_df[t].values, 0, None)))],
            verbose=False
        )

        # Predict on val in transformed space then invert
        p_va_t = model.predict(X_va)
        p_va = np.expm1(p_va_t)
        p_va = np.clip(p_va, 0, None)

        fitted[t] = model
        val_preds[t] = p_va

        # Quick metrics in original space
        y_va = np.clip(val_df[t].values, 0, None)
        mae = mean_absolute_error(y_va, p_va)
        mse  = mean_squared_error(y_va, p_va)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_val, p_va)

        metrics.append({"target": t, "MAE": mae, "RMSE": rmse, "R2": r2, "best_ntree": model.best_iteration})

    metrics_df = pd.DataFrame(metrics).sort_values("RMSE")
    return fitted, val_preds, metrics_df

In [ ]:
fitted_models, val_preds, metrics_df = fit_xgb_per_target(
    train_df, val_df, feature_cols, target_cols, seed=42
)

metrics_df

In [ ]:
preds = {}

for target in target_cols:
    pipe = fitted_models[target]
    pred = pipe.predict(ho_df[feature_cols])
    preds[target] = np.clip(np.expm1(pred), 0, None)

In [ ]:
ids = ["sample_date", "latitude", "longitude"]

pred_df = ho_df[ids].copy()
for target in target_cols:
    pred_df[target] = preds[target]

submission_df = submission_df.drop(columns=target_cols, errors="ignore").merge(pred_df, on=ids, how="left")

In [ ]:
submission_df.head(submission_df.shape[0])